In [ ]:
#pip install pandas

In [1]:
from platform import python_version
print(python_version())
import pandas as pd 
import numpy as np
import gc

3.9.13


In [6]:
Dataset_folder = "./ml-latest-small"

In [7]:
#movies = pd.read_csv(f"{Dataset_folder}/movies.csv")
#ratings = pd.read_csv(f"{Dataset_folder}/ratings.csv", chunksize=1000, usecols = ['userId','movieId', 'rating'])
#tags = pd.read_csv(f"{Dataset_folder}/tags.csv", usecols = ['userId','movieId', 'tag'])

In [8]:
#converts each users reviews to a list of tuples including the movieID and the 
def extract_users (fn):
    users = {}
    count = 0 
    header = True
    with pd.read_csv(fn, chunksize=10000) as reader:
        for chunk in reader:
            for index, row in chunk.iterrows():
                count +=1 
                if (row['userId'] not in users.keys()):
                    users [row['userId']] = [(row['movieId'], row['rating'])]
                else:
                    users [row['userId']].append((row['movieId'], row['rating']))
                if (count % 100000 == 0 ):
                    tempdict = {"userId":list(users.keys())[:-1], "ratings_list":list(users.values())[:-1]}
                    #print (tempdict)
                    tempDF = pd.DataFrame.from_dict(tempdict)

                    tempDF.to_csv("ratings_organized.csv",header=header, mode='a')

                    header = False
                    
                    for i, key in enumerate(list(users.keys())[:-1]):
                        del users [key]
                    del tempDF
                    gc.collect()
                    
    
    return users

            
    
    

In [9]:
extract_users(f"{Dataset_folder}/ratings.csv")

{610.0: [(1.0, 5.0),
  (6.0, 5.0),
  (16.0, 4.5),
  (32.0, 4.5),
  (47.0, 5.0),
  (50.0, 4.0),
  (70.0, 4.0),
  (95.0, 3.5),
  (110.0, 4.5),
  (111.0, 5.0),
  (112.0, 4.0),
  (153.0, 3.0),
  (159.0, 3.5),
  (194.0, 5.0),
  (215.0, 5.0),
  (231.0, 4.0),
  (260.0, 5.0),
  (293.0, 5.0),
  (296.0, 5.0),
  (303.0, 3.0),
  (318.0, 3.0),
  (319.0, 3.5),
  (328.0, 3.5),
  (332.0, 3.0),
  (344.0, 3.0),
  (356.0, 3.0),
  (367.0, 3.5),
  (377.0, 5.0),
  (380.0, 5.0),
  (407.0, 5.0),
  (412.0, 2.0),
  (441.0, 5.0),
  (457.0, 4.5),
  (464.0, 4.0),
  (471.0, 4.0),
  (480.0, 5.0),
  (519.0, 2.0),
  (527.0, 3.5),
  (541.0, 5.0),
  (589.0, 5.0),
  (592.0, 4.5),
  (593.0, 4.5),
  (608.0, 4.5),
  (714.0, 3.5),
  (735.0, 4.5),
  (741.0, 4.5),
  (750.0, 4.5),
  (778.0, 5.0),
  (780.0, 3.5),
  (784.0, 5.0),
  (785.0, 4.0),
  (849.0, 3.0),
  (858.0, 5.0),
  (903.0, 5.0),
  (904.0, 5.0),
  (909.0, 4.5),
  (912.0, 3.5),
  (919.0, 3.5),
  (923.0, 4.0),
  (924.0, 4.5),
  (968.0, 4.0),
  (1036.0, 5.0),
  (1047.0,

In [ ]:
#organized_reviews.to_csv("organized_reviews.csv", index=False, chunksize=10000 encoding='utf-8')

## Mesuring user simillarity:

### Case 1: Number of overlapping movies watched 

- going on a point based system we will add 100 points in the simillarity metric for every movie that both users have watched
- we must later account for movies that they both watched but had different opinions of 

### Case 2: rating similarity for movies both users have watched

- to properly mesure simillarity in taste we must also account for differences in opinions on movies two users have both watched 
- So we will subtract points based on a percentage of difference between the two users
    - simillarity = simillarity - 100* (1-(rating user1/rating user 2))
```
Ex. 
movie: shawshank redemption
user 1 rating: 1
user 2 rating: 5 
Case 1 :similarity = 100 for sharing the same film, 
Case 2: similarity = 100 - 100 *(1 - (1/5))  = 100 - 100(0.8) = 100 - 80 = 20 
new simillarity is now 20 after it has been balenced for ratings
```

### Case 3:  overall genre prefrences of the users

- We should also take into account general genre preferences as a mesure of simillarity. 
- we will also consider genre simillarity as a dividend of the percentage of each genre the user has watched 
- if 1000 points is the total afforded to each genre, simillarity will be incrmented by the following expression
- for each genre
    - similarity += 1000 * (precent of movies in certain genre for user 1/ percent for user 2)* the smaller percent from the two   users
    
    
## final step:
#### we will normalize all simillarity values between 0 and 1 using z normalization


In [17]:
from platform import python_version
print(python_version())
import pandas as pd 
import numpy as np
import gc

3.9.13


In [18]:
Dataset_folder = "./ml-latest-small"

In [19]:
Userratings = pd.read_csv(f"./ratings_organized.csv", usecols = ['userId', 'ratings_list'])

In [20]:
#Userratings

In [21]:
from pickle import dump,load

In [22]:
movies = pd.read_csv(f"{Dataset_folder}/movies.csv",usecols = ['movieId','genres'])

In [16]:
## only need to do this once 
# "banana|apple".split("|")
# genres = []
# for index, row in movies.iterrows():
#     gs = row['genres'].split("|")
#     for g in gs:
#         if (g not in genres):
#             genres.append(g)
            
# print(len(genres))
# dump(genres, open("genresList.pkl","wb"))


In [23]:
file = open("genresList.pkl",'rb')
genres = load(file)
file.close()

In [24]:
def create_genre_tracker ():
    gt = {}
    for g in genres:
        gt[g] = 0 
    return gt


In [25]:
result =movies.loc[movies['movieId'] == 1 ,'genres'].values[0]
print(result)

Adventure|Animation|Children|Comedy|Fantasy


In [26]:
import ast
def compute_simillarity_ratings (l1,l2):
    l1_dict = dict(l1)
    l2_dict = dict(l2)
    simillarity = 0 
    
    genre_tracker1 = create_genre_tracker ()
    genre_tracker2 = create_genre_tracker ()
    
    l2_keys = list(l2_dict.keys())
    for key in l1_dict.keys():
        m1_genres = movies.loc[movies['movieId'] == key  ,'genres'].values[0].split("|")
        for mg in m1_genres:
            genre_tracker1 [mg] +=1
        if key in l2_keys:
            r1 = l1_dict [key]
            r2 = l2_dict [key]
            if (r1 < r2):
                simillarity += 100*(r1/r2)
            else:
                simillarity+= 100*(r2/r1) 
                
    for key in l2_keys:
        m2_genres = movies.loc[movies['movieId'] == key  ,'genres'].values[0].split("|")
        for mg in m2_genres:
            genre_tracker2 [mg] +=1
            
    genre_simillarity =  0 
    for g in genres:
        tempg1 = genre_tracker1 [g]
        tempg2 = genre_tracker2 [g]
        if (tempg1 > 0 or tempg2 >0):
            genre_simillarity += (1000* (min(tempg1,tempg2)/max(tempg1,tempg2))* min (tempg1,tempg2))
        
    return genre_simillarity + simillarity
                
        
        

In [27]:
# num_users = len(Userratings.sample(n=100000).index)
# del Userratings
# gc.collect()
# print(num_users)

ValueError: Cannot take a larger sample than population when 'replace=False'

In [34]:
#converts each users reviews to a list of tuples including the movieID and the 
def save_simillarity (fn,numsamples =600):
    user_similarity = {}
    df = pd.read_csv(fn)   #.sample(n=numsamples)  only needed if dataset is too large
    header = True
    for index, row in df.iterrows():
        for i in range (0,len(df.index)):
            rev1 = ast.literal_eval (row["ratings_list"])
            rev2 = ast.literal_eval(df.iloc[[i]] ["ratings_list"].tolist()[0])
            if (row['userId'] not in user_similarity.keys()):
                user_similarity [row['userId']] = [compute_simillarity_ratings(rev1,rev2)]
            else:
                user_similarity [row['userId']].append(compute_simillarity_ratings(rev1,rev2))

        if (index % 10 == 0 or index == len(df.index)-1 ):
            print(index)
            tempdict = {"userId":list(user_similarity.keys())[:-1], "simillarity_vector":list(user_similarity.values())[:-1]}
            #print (tempdict)
            tempDF = pd.DataFrame.from_dict(tempdict)

            tempDF.to_csv("simillarity_matrix.csv",header=header, mode='a')

            header = False

            for i, key in enumerate(list(user_similarity.keys())[:-1]):
                del user_similarity [key]
            del tempDF
            gc.collect()

    
    return

In [36]:
save_simillarity(f"./ratings_organized.csv")

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
608


In [ ]:
print(type([1,2,3]))

In [50]:
sim_df =  pd.read_csv("./simillarity_matrix.csv")
all_vals = []

for index, row in sim_df.iterrows():
    sim_list = ast.literal_eval(row["simillarity_vector"])
    all_vals += sim_list
    
print (len(all_vals))
max_sim = max(all_vals)
min_Sim = min(all_vals)

print(min_Sim)
print(max_sim)

370272
898.0902248084436
6885800.0


In [46]:
def z_normalization (max_Val,min_Val, val):
    return (val-min_Val)/(max_Val - min_Val)

In [55]:
for index, row in sim_df.iterrows():
    sim_list = ast.literal_eval(row["simillarity_vector"])
    newlist = []
    for sim in sim_list:
        newlist.append(z_normalization(max_sim,min_Sim,sim))
    sim_df.at[index,"simillarity_vector"] = newlist
    
    

ValueError: malformed node or string: [0.1044752589363584, 0.0015251954457422966, 0.0031721047843374524, 0.032986264162417234, 0.004299825532620717, 0.05187579253985946, 0.04553496196028419, 0.004720593648712787, 0.002965655905863573, 0.028673362455182, 0.010471745892390706, 0.0036063684845653227, 0.001868213292917232, 0.004338253034008866, 0.03744789576784438, 0.01797313264847442, 0.021906123544926964, 0.05306583785226225, 0.04424733612658569, 0.06064926827880598, 0.05323612640607357, 0.027100583406729665, 0.025890208899284943, 0.027445768108451733, 0.0016135557400262983, 0.0011346972817110094, 0.03796379474599319, 0.043341218554838815, 0.015920206264042305, 0.0032773307060657343, 0.0050355596625045055, 0.018557209036942936, 0.02088795104470723, 0.017534048269970223, 0.0010500324854360735, 0.0068697297320740435, 0.0009827008786108616, 0.01241427255092249, 0.02227690651371394, 0.01754003620339781, 0.04470854989083392, 0.052429586472866246, 0.02499551026642876, 0.004416369599796494, 0.0

In [57]:
sim_df.to_csv("simillarity_matrix_normalized.csv",index=False)